In [1]:
from known_boundary.GP import optimise,optimise_warp,optimise_warp_no_boundary
from known_boundary.utlis import Trans_function, get_initial_points,transform
from known_boundary.acquisition_function import EI_acquisition_opt,MES_acquisition_opt,Warped_EI_acquisition_opt,Warped_TEI2_acquisition_opt,Warped_TEI1_acquisition_opt,LCB_acquisition_opt,ERM_acquisition_opt
import numpy as np
import GPy
import torch
from botorch.test_functions import Ackley,Levy,Beale,Branin,Hartmann,Rosenbrock,Powell
from botorch.utils.transforms import unnormalize,normalize

import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double

function_information = []


# temp={}
# temp['name']='Branin2D' 
# temp['function'] = Branin(negate=False)
# temp['fstar'] =  0.397887
# temp['min']=True 
# function_information.append(temp)

# temp={}
# temp['name']='Beale2D' 
# temp['function'] = Beale(negate=False)
# temp['fstar'] =  0. 
# temp['min']=True 
# function_information.append(temp)

# temp={}
# temp['name']='Hartmann3D' 
# temp['function'] = Hartmann(dim=3,negate=False)
# temp['fstar'] =  -3.86278 
# temp['min']=True 
# function_information.append(temp)


temp={}
temp['name']='Powell4D' 
temp['function'] = Powell(dim=4,negate=False)
temp['fstar'] = 0. 
temp['min']=True 
function_information.append(temp)

# temp={}
# temp['name']='Rosenbrock5D' 
# temp['function'] = Rosenbrock(dim=5,negate=False)
# temp['fstar'] = 0.
# temp['min']=True 
# function_information.append(temp)

# temp={}
# temp['name']='Ackley7D' 
# temp['function'] = Ackley(dim=7,negate=False)
# temp['fstar'] = 0.
# temp['min']=True 
# function_information.append(temp)


/home/hanyang/anaconda3/envs/known_boundary/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for information in function_information:
    fun = information['function']
    dim = fun.dim
    bounds = fun.bounds
    standard_bounds=np.array([0.,1.]*dim).reshape(-1,2)

    n_init = 4*dim
    iter_num = 15*dim 
    N = 20

    fstar = information['fstar']
    fun = Trans_function(fun,fstar,min=True)

In [5]:
Warped_BO_logTEI = []

for exp in range(N):

    seed = 99 - exp
    
    print(exp)

    X_BO = get_initial_points(bounds, n_init,device,dtype,seed=seed)
    Y_BO = torch.tensor(
        [fun(x) for x in X_BO], dtype=dtype, device=device
    ).reshape(-1,1)



    best_record = [Y_BO.min().item()]
    np.random.seed(1234)

    for i in range(iter_num):

            print('inner loop: ',i)
        
            train_Y = Y_BO.numpy()
            train_X = normalize(X_BO, bounds)
            train_X = train_X.numpy()
            
            # train the GP
            if i%1 == 0:
                res = optimise_warp(train_X, train_Y)
                lengthscale = np.sqrt(res[0])
                variance = res[1]
                c = res[2]
            
            
            warp_Y = np.log(train_Y+c)
            mean_warp_Y = np.mean(warp_Y) # use to predict mean
            warp_Y_standard = warp_Y-mean_warp_Y
            
            
            kernel = GPy.kern.RBF(input_dim=dim,lengthscale= lengthscale,variance=variance)  
            m = GPy.models.GPRegression(train_X, warp_Y_standard,kernel)
            m.Gaussian_noise.variance.fix(10**(-5))
            
            standard_next_X = Warped_TEI2_acquisition_opt(model=m,bounds=standard_bounds,f_best=best_record[-1],c=c,f_mean=mean_warp_Y)
            X_next = unnormalize(torch.tensor(standard_next_X), bounds).reshape(-1,dim)            
            Y_next = fun(X_next).reshape(-1,1)

            print(standard_next_X)
            # Append data
            X_BO = torch.cat((X_BO, X_next), dim=0)
            Y_BO = torch.cat((Y_BO, Y_next), dim=0)
            
            best_record.append(Y_BO.min().item())
            print(best_record[-1])
            
    best_record = np.array(best_record)+fstar         
    Warped_BO_logTEI.append(best_record)

0
inner loop:  0
[0.13774541 0.46292351 0.55927758 0.04804909]
124.3501590735082
inner loop:  1
[0.12514804 0.32152735 0.51002415 0.        ]
124.3501590735082
inner loop:  2
[0.16149668 0.54269854 0.62873981 0.09422449]
124.3501590735082
inner loop:  3
[0.17153942 0.51693134 0.46605542 0.05526319]
96.88782056363135
inner loop:  4
[0.09215389 0.50485565 0.44404298 0.13081116]
45.111421207548446
inner loop:  5
[0.         0.51732439 0.41056478 0.16532146]
45.111421207548446
inner loop:  6
[0.12996572 0.4617006  0.46333073 0.19734405]
31.63808207445336
inner loop:  7
[0.1503084  0.41986356 0.39630045 0.20234134]
31.63808207445336
inner loop:  8
[0.11980836 0.41440617 0.4980718  0.2203351 ]
31.63808207445336
inner loop:  9
[0.15389122 0.51046541 0.42788618 0.22519186]
30.037170715530987
inner loop:  10
[1. 0. 0. 0.]
30.037170715530987
inner loop:  11
[0. 1. 0. 1.]
30.037170715530987
inner loop:  12
[0.6144747 0.        0.        1.       ]
30.037170715530987
inner loop:  13
[0.14930436 0.

KeyboardInterrupt: 

In [12]:
best_record

array([3.54519686, 3.54519686, 2.31585921, 0.90380293, 0.48517651,
       0.48517651, 0.48517651, 0.43108345, 0.43108345, 0.43108345,
       0.43108345, 0.43108345, 0.43108345, 0.43108345, 0.43108345,
       0.43108345, 0.43108345, 0.43108345, 0.43108345, 0.43108345,
       0.43108345, 0.43108345, 0.43108345, 0.43108345, 0.43108345,
       0.43108345, 0.43108345, 0.43108345, 0.43108345, 0.43108345,
       0.40693281])

In [4]:
best_record

array([3.54519686, 3.54519686, 2.31585676, 0.90380406, 0.48517266,
       0.48517266, 0.48517266, 0.43107989, 0.43107989, 0.43107989,
       0.43107989, 0.43107989, 0.43107989, 0.43107989, 0.43107989,
       0.43107989, 0.43107989, 0.43107989, 0.43107989, 0.43107989,
       0.43107989, 0.43107989, 0.43107989, 0.43107989, 0.43107989,
       0.43107989, 0.43107989, 0.43107989, 0.43107989, 0.43107989,
       0.40693314])

In [8]:
################################## logGP+EI (general boundary) ######################################
    
logBO_general_boundary_logTEI = []

for exp in range(1):
    
    print(exp)

    seed = 0

    X_BO = get_initial_points(bounds, n_init,device,dtype,seed=seed)
    Y_BO = torch.tensor(
        [fun(x) for x in X_BO], dtype=dtype, device=device
    ).reshape(-1,1)

    best_record = [Y_BO.min().item()]

    np.random.seed(1234)

    for i in range(60):
            
            #print(i)
            
            train_Y = (Y_BO - Y_BO.mean()) / Y_BO.std()
            train_X = normalize(X_BO, bounds)
            
            fstar_temp = 0
            fstar_standard = (fstar_temp - Y_BO.mean()) / Y_BO.std()
            fstar_standard = fstar_standard.item()
            
            train_Y = train_Y.numpy()
            train_X = train_X.numpy()
            
            standard_best_record = [np.min(train_Y)]
            
            
            # train the GP
            if i%1 == 0:
                delta2 = 0.3
                delta2_standard = delta2/ Y_BO.std()
                
                res = optimise_warp_no_boundary(train_X, train_Y,-fstar_standard+delta2_standard)
                lengthscale = np.sqrt(res[0])
                variance = res[1]
                c = res[2]
            
            print('lengthscale is: ', lengthscale)
            print('variance is: ',variance)
            # print('c is: ',c)
            # print('check:',-np.min(train_Y)+10**(-5),-np.min(train_Y)-fstar_standard+1.5)
            
            warp_Y = np.log(train_Y+c)
            mean_warp_Y = np.mean(warp_Y) # use to predict mean
            warp_Y_standard = warp_Y-mean_warp_Y
            
            
            kernel = GPy.kern.RBF(input_dim=dim,lengthscale= lengthscale,variance=variance)  #np.sqrt(res[0])
            m = GPy.models.GPRegression(train_X, warp_Y_standard,kernel)
            m.Gaussian_noise.variance.fix(10**(-5))
            
            c_unstandard = -c*Y_BO.std()+Y_BO.mean()
            print('C is: ',c_unstandard)
            if c_unstandard>=fstar_temp:
                print('logEI')
                standard_next_X = Warped_EI_acquisition_opt(model=m,bounds=standard_bounds,f_best=standard_best_record[-1],c=c,f_mean=mean_warp_Y)
            else:
                print('logTEI')
                standard_next_X = Warped_TEI1_acquisition_opt(model=m,bounds=standard_bounds,f_best=standard_best_record[-1],c=c,f_mean=mean_warp_Y,fstar=fstar_standard)
            
            
            #standard_next_X = Warped_EI_acquisition_opt(model=m,bounds=standard_bounds,f_best=standard_best_record[-1],c=c,f_mean=mean_warp_Y)
            X_next = unnormalize(torch.tensor(standard_next_X), bounds).reshape(-1,dim)            
            Y_next = fun(X_next).reshape(-1,1)

            # Append data
            X_BO = torch.cat((X_BO, X_next), dim=0)
            Y_BO = torch.cat((Y_BO, Y_next), dim=0)
            
            best_record.append(Y_BO.min().item())
            print(best_record[-1])
            
    best_record = np.array(best_record) +fstar          
    logBO_general_boundary_logTEI.append(best_record)
        
        


0
lengthscale is:  0.258506476894459
variance is:  2.1667706270008815
C is:  tensor(98.5747, dtype=torch.float64)
logEI
90.3208597456099
lengthscale is:  0.24534085585191348
variance is:  2.205513651885198
C is:  tensor(-0.3000, dtype=torch.float64)
logTEI
56.6832243449538
lengthscale is:  0.2600487020310177
variance is:  2.507281403990475
C is:  tensor(-0.3000, dtype=torch.float64)
logTEI
47.11564925951778
lengthscale is:  0.2852573388227651
variance is:  2.7791893450867673
C is:  tensor(-0.3000, dtype=torch.float64)
logTEI
47.11564925951778
lengthscale is:  0.23597498220578764
variance is:  2.8644749042832527
C is:  tensor(-0.3000, dtype=torch.float64)
logTEI
6.108001427617168
lengthscale is:  0.19019326219480903
variance is:  4.5801652138855635
C is:  tensor(-0.3000, dtype=torch.float64)
logTEI
6.108001427617168
lengthscale is:  0.10957998787207197
variance is:  4.035967586686327
C is:  tensor(-0.3000, dtype=torch.float64)
logTEI
6.108001427617168
lengthscale is:  0.1120554114825443

In [ ]:
logBO_general_boundary_logTEI

In [17]:
information['name']

'Branin2D'

In [24]:
np.savetxt('exp_res2/'+information['name']+'_logGP+logTEI(general boundary)', logBO_general_boundary_logTEI, delimiter=',')

In [18]:
################################## logGP+EI (no boundary) ######################################
    
logBO_no_boundary_logEI = []

for exp in range(1):
    
    print(exp)

    seed = exp

    X_BO = get_initial_points(bounds, n_init,device,dtype,seed=seed)
    Y_BO = torch.tensor(
        [fun(x) for x in X_BO], dtype=dtype, device=device
    ).reshape(-1,1)

    best_record = [Y_BO.min().item()]

    np.random.seed(1234)

    for i in range(20):
            
            #print(i)
            
            train_Y = Y_BO
            train_X = normalize(X_BO, bounds)

            fstar_standard = 0.
            
            train_Y = train_Y.numpy()
            train_X = train_X.numpy()
            
            standard_best_record = [np.min(train_Y)]
            
            
            # train the GP
            if i%1 == 0:
                
                res = optimise_warp_no_boundary(train_X, train_Y,-fstar_standard+4*Y_BO.std().item())
                lengthscale = np.sqrt(res[0])
                variance = res[1]
                c = res[2]
            
            
            print('l is: ',lengthscale)
            print('variance is: ', variance)
            
            warp_Y = np.log(train_Y+c)
            mean_warp_Y = np.mean(warp_Y) # use to predict mean
            warp_Y_standard = warp_Y-mean_warp_Y
            
            print(mean_warp_Y)
            
            
            kernel = GPy.kern.RBF(input_dim=dim,lengthscale= lengthscale,variance=variance)  #np.sqrt(res[0])
            m = GPy.models.GPRegression(train_X, warp_Y_standard,kernel)
            m.Gaussian_noise.variance.fix(10**(-5))
            
            print('C is: ',-c)
            
            standard_next_X = Warped_EI_acquisition_opt(model=m,bounds=standard_bounds,f_best=standard_best_record[-1],c=c,f_mean=mean_warp_Y)
            X_next = unnormalize(torch.tensor(standard_next_X), bounds).reshape(-1,dim)            
            Y_next = fun(X_next).reshape(-1,1)

            # Append data
            X_BO = torch.cat((X_BO, X_next), dim=0)
            Y_BO = torch.cat((Y_BO, Y_next), dim=0)
            
            best_record.append(Y_BO.min().item())
            print(best_record[-1])
            
    best_record = np.array(best_record) +fstar          
    logBO_no_boundary_logEI.append(best_record)
        

0
l is:  0.2585068433832803
variance is:  2.166765796112884
8.025056075812774
C is:  98.57309786747382
90.31607169394279
l is:  0.28167357897844375
variance is:  1.8274767625178856
8.021448913424619
C is:  -116.15461862240026
60.87985575028843
l is:  0.2886993751420986
variance is:  2.199149512802875
7.770799645252795
C is:  -45.51979509277098
5.004603584396312
l is:  0.30963608031980294
variance is:  2.27681887927007
7.699472871556965
C is:  -114.48113623705252
5.004603584396312
l is:  0.32849697540397427
variance is:  2.322544272912786
7.6118938319593425
C is:  -141.4852581402147
5.004603584396312
l is:  0.3644347751561766
variance is:  2.4214508008046813
7.574713753236052
C is:  -192.73499857923235
5.004603584396312
l is:  0.40891332293866184
variance is:  2.962844162095975
7.502669884922318
C is:  -209.01517038114642
5.004603584396312
l is:  0.418889910738949
variance is:  2.9837479578163246
7.467876071312194
C is:  -214.76786783592638
5.004603584396312
l is:  0.43152947147140014
v

In [15]:
np.savetxt('exp_res2/'+information['name']+'_logGP(NoBoundary)+logEI', logBO_no_boundary_logEI, delimiter=',')